reading data

In [1]:
# ===============================================================
# NYC Yellow Taxi Trip Data Cleansing - February 2025
# Enhanced Fare Validation
# ===============================================================

import pandas as pd
import numpy as np  
import os

# ---------------------------------------------------------------
# 1. Setup
# ---------------------------------------------------------------

file_path = os.path.join(os.path.expanduser("~"), "Downloads", "yellow_tripdata_2025-02.parquet")
df = pd.read_parquet(file_path)

print("==============================================================")
print(" NYC Taxi Trip Data Cleansing - February 2025")
print("==============================================================\n")

initial_rows = len(df)
print(f"Initial rows loaded: {initial_rows:,}")

 NYC Taxi Trip Data Cleansing - February 2025

Initial rows loaded: 3,577,543


CleaningReport

In [2]:
cleaning_report = []

def log_step(step_name, before, after):
    cleaning_report.append({
        "Step": step_name,
        "Rows Before": before,
        "Rows After": after,
        "Rows Dropped": before - after,
        "Remaining %": round(after / initial_rows * 100, 2)
    })
    print(f"[{step_name}] - Dropped {before - after:,} rows "
          f"({round((before - after)/before*100,2) if before>0 else 0:.2f}%)")

def validate_column(df, condition, col_name, description=""):
    invalid_count = (~condition).sum()
    print(f"{col_name}: {invalid_count:,} invalid {description}")
    return df[condition].copy(), invalid_count

Output of general informations

In [3]:
# Display basic information about the dataframe
# print first 5 rows, info, count, and descriptive statistics
print("Basic Information about the DataFrame:")
print("First 5 rows:")
print(df.head(5))
print("*******************************************")
print("Info:")
print(df.info())
print("*******************************************")
print("Count:")
print(df.count())
print("*******************************************")
print("Descriptive Statistics:")
print(df.describe())
print("*******************************************")

Basic Information about the DataFrame:
First 5 rows:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2025-02-01 00:12:18   2025-02-01 00:32:33              3.0   
1         2  2025-02-01 00:40:04   2025-02-01 00:49:15              1.0   
2         1  2025-02-01 00:06:09   2025-02-01 00:11:51              0.0   
3         1  2025-02-01 00:15:13   2025-02-01 00:20:19              0.0   
4         2  2025-02-01 00:02:52   2025-02-01 00:20:25              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.12         1.0                  N           246            79   
1           1.40         1.0                  N           114            79   
2           0.40         1.0                  N           211           144   
3           0.70         1.0                  N           113           249   
4           4.19         1.0                  N           113           263   

   payment_type  fare

STARTING CLEANSING

In [4]:
# Initial number of rows in the dataframe
initial_row_count = len(df)
print(f"Initial Number of rows in the dataframe: {initial_row_count:,.0f}")
# print STARTING CLEANSING
print("===== STARTING CLEANSING =====")
print("*******************************************")

Initial Number of rows in the dataframe: 3,577,543
===== STARTING CLEANSING =====
*******************************************


VendorID

In [5]:
# Column VendorID allowed values
allowed_vendor_ids = [1, 2, 6, 7]

# Create boolean mask for invalid VendorIDs
invalid_vendor = ~df['VendorID'].isin(allowed_vendor_ids)
invalid_count = invalid_vendor.sum()

if invalid_count > 0:
    print("=== VendorID Validation Report ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"Invalid VendorID entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Show which invalid VendorIDs were found
    invalid_values = df.loc[invalid_vendor, 'VendorID'].value_counts().sort_index()
    print(f"\nInvalid VendorID values found:")
    for vendor_id, count in invalid_values.items():
        print(f"  VendorID {vendor_id}: {count:,} rows")
    
    # Drop invalid VendorIDs
    df = df[~invalid_vendor].copy()
    print(f"\nDropped invalid VendorID entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("No invalid VendorID entries found.")

print("*******************************************")

No invalid VendorID entries found.
*******************************************


Dates (tpep_pickup_datetime & tpep_dropoff_datetime)

In [6]:
# Definition: tpep_dropoff_datetime determines the month of the trip/payment, should be February 2025!

# Convert datetime columns once (efficient approach)
pickup_dt = pd.to_datetime(df['tpep_pickup_datetime'], errors='coerce')
dropoff_dt = pd.to_datetime(df['tpep_dropoff_datetime'], errors='coerce')

# Create boolean masks for each validation check
invalid_pickup = pickup_dt.isna()
invalid_dropoff = dropoff_dt.isna()

# Check for rows where order needs to be swapped
valid_both = ~invalid_pickup & ~invalid_dropoff
needs_swap = valid_both & (dropoff_dt <= pickup_dt)
swap_count = needs_swap.sum()

# Swap pickup and dropoff where needed
if swap_count > 0:
    print(f"Found {swap_count:,} rows with incorrect order - swapping pickup/dropoff times")
    pickup_temp = pickup_dt.copy()
    pickup_dt = pickup_dt.where(~needs_swap, dropoff_dt)
    dropoff_dt = dropoff_dt.where(~needs_swap, pickup_temp)

# Now check date range on corrected dropoff times
invalid_month = valid_both & ((dropoff_dt.dt.month != 2) | (dropoff_dt.dt.year != 2025))

# Combine invalid conditions (excluding order issues since we fixed them)
all_invalid = invalid_pickup | invalid_dropoff | invalid_month
invalid_count = all_invalid.sum()

if invalid_count > 0 or swap_count > 0:
    print("\n=== Datetime Validation Report ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"Invalid tpep_pickup_datetime rows: {invalid_pickup.sum():,}")
    print(f"Invalid tpep_dropoff_datetime rows: {invalid_dropoff.sum():,}")
    print(f"Rows with swapped pickup/dropoff: {swap_count:,}")
    print(f"Rows where dropoff not in February 2025: {invalid_month.sum():,}")
    print(f"\nTotal rows dropped: {invalid_count:,}")
    print(f"Percentage dropped: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid rows and update datetime columns
    df = df[~all_invalid].copy()
    df['tpep_pickup_datetime'] = pickup_dt[~all_invalid]
    df['tpep_dropoff_datetime'] = dropoff_dt[~all_invalid]
    
    print(f"\nRows remaining: {len(df):,}")
else:
    print("No invalid datetime entries found.")
    # Still convert to datetime dtype for consistency
    df['tpep_pickup_datetime'] = pickup_dt
    df['tpep_dropoff_datetime'] = dropoff_dt

print("*******************************************")

# Calculate trip duration in minutes
df['trip_duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Show duration statistics BEFORE filtering
print("\n=== Trip Duration Statistics (Before Filtering) ===")
print(f"Mean duration: {df['trip_duration_minutes'].mean():.2f} minutes")
print(f"Median duration: {df['trip_duration_minutes'].median():.2f} minutes")
print(f"Min duration: {df['trip_duration_minutes'].min():.2f} minutes")
print(f"Max duration: {df['trip_duration_minutes'].max():.2f} minutes")
print(f"Std deviation: {df['trip_duration_minutes'].std():.2f} minutes")

# Count trips with extreme durations
short_trips_count = (df['trip_duration_minutes'] < 1).sum()
long_trips_count = (df['trip_duration_minutes'] > 300).sum()

print(f"\nTrips with duration < 1 minute: {short_trips_count:,} ({(short_trips_count / len(df)) * 100:.2f}%)")
print(f"Trips with duration > 5 hours: {long_trips_count:,} ({(long_trips_count / len(df)) * 100:.2f}%)")

# Filter out extreme durations
extreme_duration = (df['trip_duration_minutes'] < 1) | (df['trip_duration_minutes'] > 300)
extreme_count = extreme_duration.sum()

if extreme_count > 0:
    print(f"\n=== Extreme Duration Validation ===")
    print(f"Total trips with extreme duration (< 1 min or > 5 hours): {extreme_count:,}")
    print(f"Percentage: {(extreme_count / initial_row_count) * 100:.2f}%")
    
    # Drop extreme duration trips
    df = df[~extreme_duration].copy()
    
    print(f"Dropped extreme duration trips.")
    print(f"Rows remaining: {len(df):,}")
    
    # Show updated statistics AFTER filtering
    print(f"\n=== Trip Duration Statistics (After Filtering) ===")
    print(f"Mean duration: {df['trip_duration_minutes'].mean():.2f} minutes")
    print(f"Median duration: {df['trip_duration_minutes'].median():.2f} minutes")
    print(f"Min duration: {df['trip_duration_minutes'].min():.2f} minutes")
    print(f"Max duration: {df['trip_duration_minutes'].max():.2f} minutes")
    print(f"Std deviation: {df['trip_duration_minutes'].std():.2f} minutes")
else:
    print("\nNo extreme duration trips found.")

print("*******************************************")

Found 5,164 rows with incorrect order - swapping pickup/dropoff times

=== Datetime Validation Report ===
Rows before validation: 3,577,543
Invalid tpep_pickup_datetime rows: 0
Invalid tpep_dropoff_datetime rows: 0
Rows with swapped pickup/dropoff: 5,164
Rows where dropoff not in February 2025: 2,406

Total rows dropped: 2,406
Percentage dropped: 0.07%

Rows remaining: 3,575,137
*******************************************

=== Trip Duration Statistics (Before Filtering) ===
Mean duration: 15.39 minutes
Median duration: 12.20 minutes
Min duration: 0.00 minutes
Max duration: 8564.67 minutes
Std deviation: 26.19 minutes

Trips with duration < 1 minute: 40,229 (1.13%)
Trips with duration > 5 hours: 1,132 (0.03%)

=== Extreme Duration Validation ===
Total trips with extreme duration (< 1 min or > 5 hours): 41,361
Percentage: 1.16%
Dropped extreme duration trips.
Rows remaining: 3,533,776

=== Trip Duration Statistics (After Filtering) ===
Mean duration: 15.17 minutes
Median duration: 12.32 

passenger_count

In [7]:
# Invalid passenger_count: NaN or <= 0 or > 10
# Create boolean masks for each validation check
is_nan = df['passenger_count'].isna()
is_zero_or_negative = df['passenger_count'] <= 0
is_too_high = df['passenger_count'] > 10

# Combine all invalid conditions
all_invalid = is_nan | is_zero_or_negative | is_too_high
invalid_count = all_invalid.sum()

if invalid_count > 0:
    print("=== Passenger Count Validation Report ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"NaN passenger_count rows: {is_nan.sum():,}")
    print(f"Passenger_count <= 0 rows: {is_zero_or_negative.sum():,}")
    print(f"Passenger_count > 10 rows: {is_too_high.sum():,}")
    print(f"\nTotal invalid passenger_count entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Set invalid passenger_count values to default of 1
    df.loc[all_invalid, 'passenger_count'] = 1
    
    print(f"\nSet invalid passenger_count values to default of 1.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("No invalid passenger_count entries found.")

print("*******************************************")

=== Passenger Count Validation Report ===
Rows before validation: 3,533,776
NaN passenger_count rows: 803,554
Passenger_count <= 0 rows: 21,060
Passenger_count > 10 rows: 0

Total invalid passenger_count entries: 824,614
Percentage of invalid entries: 23.05%

Set invalid passenger_count values to default of 1.
Rows remaining: 3,533,776
*******************************************


trip_distance

In [8]:
# Invalid trip_distance: NaN or <= 0 or > 500
# Create boolean masks for each validation check
is_nan = df['trip_distance'].isna()
is_negative = df['trip_distance'] < 0
is_zero = df['trip_distance'] == 0
is_too_high = df['trip_distance'] > 500

# Combine all invalid conditions
all_invalid = is_nan | is_negative | is_zero | is_too_high
invalid_count = all_invalid.sum()

if invalid_count > 0:
    print("=== Trip Distance Validation Report ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"Negative trip_distance rows: {is_negative.sum():,}")
    print(f"Zero trip_distance rows: {is_zero.sum():,}")
    print(f"NaN trip_distance rows: {is_nan.sum():,}")
    print(f"Trip_distance > 500 rows: {is_too_high.sum():,}")
    print(f"\nTotal invalid trip_distance entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid trip_distance entries
    df = df[~all_invalid].copy()
    
    print(f"\nDropped invalid trip_distance entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("No invalid trip_distance entries found.")

print("*******************************************")

=== Trip Distance Validation Report ===
Rows before validation: 3,533,776
Negative trip_distance rows: 0
Zero trip_distance rows: 75,521
NaN trip_distance rows: 0
Trip_distance > 500 rows: 141

Total invalid trip_distance entries: 75,662
Percentage of invalid entries: 2.11%

Dropped invalid trip_distance entries.
Rows remaining: 3,458,114
*******************************************


RatecodeID
correct rate code might be possible to be derifed from other data, but for simplicity we set to 1

In [9]:
# Column RatecodeID allowed values
allowed_ratecode_ids = [1, 2, 3, 4, 5, 6, 99]

# Check for NaN values first and set to default
nan_count = df['RatecodeID'].isna().sum()
if nan_count > 0:
    print("=== RatecodeID Missing Values ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"NaN/Missing RatecodeID: {nan_count:,} rows ({(nan_count / len(df)) * 100:.2f}%)")
    df['RatecodeID'] = df['RatecodeID'].fillna(1)
    print(f"Set missing RatecodeID values to default of 1 (Standard rate).\n")
    
# Create boolean mask for invalid RatecodeIDs (after filling NaN)
invalid_ratecode = ~df['RatecodeID'].isin(allowed_ratecode_ids)
invalid_count = invalid_ratecode.sum()

# Show overview of all RatecodeID values
print("=== RatecodeID Overview ===")
print(f"Total rows: {len(df):,}")
print(f"\nValid RatecodeID distribution:")
valid_values = df.loc[~invalid_ratecode, 'RatecodeID'].value_counts().sort_index()
for ratecode_id, count in valid_values.items():
    percentage = (count / len(df)) * 100
    print(f"  RatecodeID {ratecode_id}: {count:,} rows ({percentage:.2f}%)")

if invalid_count > 0:
    print(f"\n=== Invalid RatecodeID Values ===")
    print(f"Invalid RatecodeID entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Show which invalid RatecodeIDs were found
    invalid_values = df.loc[invalid_ratecode, 'RatecodeID'].value_counts().sort_index()
    print(f"\nInvalid RatecodeID values found:")
    for ratecode_id, count in invalid_values.items():
        percentage = (count / len(df)) * 100
        print(f"  RatecodeID {ratecode_id}: {count:,} rows ({percentage:.2f}%)")
    
    # Drop invalid RatecodeIDs
    df = df[~invalid_ratecode].copy()
    
    print(f"\nDropped invalid RatecodeID entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("\nNo invalid RatecodeID entries found.")

print("*******************************************")

=== RatecodeID Missing Values ===
Rows before validation: 3,458,114
NaN/Missing RatecodeID: 741,846 rows (21.45%)
Set missing RatecodeID values to default of 1 (Standard rate).

=== RatecodeID Overview ===
Total rows: 3,458,114

Valid RatecodeID distribution:
  RatecodeID 1.0: 3,319,390 rows (95.99%)
  RatecodeID 2.0: 77,739 rows (2.25%)
  RatecodeID 3.0: 7,119 rows (0.21%)
  RatecodeID 4.0: 6,331 rows (0.18%)
  RatecodeID 5.0: 10,864 rows (0.31%)
  RatecodeID 99.0: 36,671 rows (1.06%)

No invalid RatecodeID entries found.
*******************************************


store_and_fwd_flag

In [10]:
# Column store_and_fwd_flag allowed values
allowed_store_and_fwd_flags = ['Y', 'N']

# Check for NaN values first and set to default
nan_count = df['store_and_fwd_flag'].isna().sum()
if nan_count > 0:
    print("=== Store and Forward Flag Missing Values ===")
    print(f"Rows before validation: {len(df):,}")
    print(f"NaN/Missing store_and_fwd_flag: {nan_count:,} rows ({(nan_count / len(df)) * 100:.2f}%)")
    df['store_and_fwd_flag'] = df['store_and_fwd_flag'].fillna('N')
    print(f"Set missing store_and_fwd_flag values to default of 'N'.\n")

# Create boolean mask for invalid store_and_fwd_flag (after filling NaN)
invalid_flag = ~df['store_and_fwd_flag'].isin(allowed_store_and_fwd_flags)
invalid_count = invalid_flag.sum()

# Show overview of all store_and_fwd_flag values
print("=== Store and Forward Flag Overview ===")
print(f"Total rows: {len(df):,}")
print(f"\nValid store_and_fwd_flag distribution:")
valid_values = df.loc[~invalid_flag, 'store_and_fwd_flag'].value_counts().sort_index()
for flag, count in valid_values.items():
    percentage = (count / len(df)) * 100
    print(f"  Flag '{flag}': {count:,} rows ({percentage:.2f}%)")

if invalid_count > 0:
    print(f"\n=== Invalid Store and Forward Flag Values ===")
    print(f"Invalid store_and_fwd_flag entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Show which invalid flags were found
    invalid_values = df.loc[invalid_flag, 'store_and_fwd_flag'].value_counts().sort_index()
    print(f"\nInvalid store_and_fwd_flag values found:")
    for flag, count in invalid_values.items():
        percentage = (count / len(df)) * 100
        print(f"  Flag '{flag}': {count:,} rows ({percentage:.2f}%)")
    
    # Drop invalid store_and_fwd_flag entries
    df = df[~invalid_flag].copy()
    
    print(f"\nDropped invalid store_and_fwd_flag entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("\nNo invalid store_and_fwd_flag entries found.")

print("*******************************************")

=== Store and Forward Flag Missing Values ===
Rows before validation: 3,458,114
NaN/Missing store_and_fwd_flag: 741,846 rows (21.45%)
Set missing store_and_fwd_flag values to default of 'N'.

=== Store and Forward Flag Overview ===
Total rows: 3,458,114

Valid store_and_fwd_flag distribution:
  Flag 'N': 3,430,730 rows (99.21%)
  Flag 'Y': 27,384 rows (0.79%)

No invalid store_and_fwd_flag entries found.
*******************************************


LocationID (PULocationID & DOLocationID)

In [11]:
# LocationID columns allowed values: NYC Taxi Zone IDs range from 1 to 265

# ===== PULocationID Validation =====
print("=== PULocationID (Pickup Location) Validation ===")
print(f"Rows before validation: {len(df):,}")
# Check for NaN values first
nan_pu_count = df['PULocationID'].isna().sum()
if nan_pu_count > 0:
    print(f"NaN/Missing PULocationID: {nan_pu_count:,} rows ({(nan_pu_count / len(df)) * 100:.2f}%)")
    # Drop rows with NaN PULocationID (cannot assume a default pickup location)
    df = df[df['PULocationID'].notna()].copy()
    print(f"Dropped missing PULocationID entries.\n")

# Create boolean mask for invalid PULocationIDs (outside range 1-265)
invalid_pu = (df['PULocationID'] < 1) | (df['PULocationID'] > 265)
invalid_pu_count = invalid_pu.sum()

# Show top 10 most common valid pickup locations
valid_pu = ~invalid_pu
print(f"Top 10 pickup locations:")
top_pu = df.loc[valid_pu, 'PULocationID'].value_counts().head(10)
for location_id, count in top_pu.items():
    percentage = (count / len(df)) * 100
    print(f"  LocationID {int(location_id)}: {count:,} rows ({percentage:.2f}%)")

if invalid_pu_count > 0:
    print(f"\n=== Invalid PULocationID Values ===")
    print(f"Invalid PULocationID entries: {invalid_pu_count:,}")
    print(f"Percentage of invalid entries: {(invalid_pu_count / initial_row_count) * 100:.2f}%")
    
    # Show invalid values (limited to first 20)
    invalid_pu_values = df.loc[invalid_pu, 'PULocationID'].value_counts().sort_index().head(20)
    print(f"\nInvalid PULocationID values found (showing up to 20):")
    for location_id, count in invalid_pu_values.items():
        print(f"  LocationID {location_id}: {count:,} rows")
    
    # Drop invalid PULocationIDs
    df = df[~invalid_pu].copy()
    print(f"\nDropped invalid PULocationID entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("\nNo invalid PULocationID entries found.")

print("*******************************************")

# ===== DOLocationID Validation =====
print("=== DOLocationID (Dropoff Location) Validation ===")

# Check for NaN values first
nan_do_count = df['DOLocationID'].isna().sum()
if nan_do_count > 0:
    print(f"NaN/Missing DOLocationID: {nan_do_count:,} rows ({(nan_do_count / len(df)) * 100:.2f}%)")
    # Drop rows with NaN DOLocationID (cannot assume a default dropoff location)
    df = df[df['DOLocationID'].notna()].copy()
    print(f"Dropped missing DOLocationID entries.\n")

# Create boolean mask for invalid DOLocationIDs (outside range 1-265)
invalid_do = (df['DOLocationID'] < 1) | (df['DOLocationID'] > 265)
invalid_do_count = invalid_do.sum()

# Show top 10 most common valid dropoff locations
valid_do = ~invalid_do
print(f"Top 10 dropoff locations:")
top_do = df.loc[valid_do, 'DOLocationID'].value_counts().head(10)
for location_id, count in top_do.items():
    percentage = (count / len(df)) * 100
    print(f"  LocationID {int(location_id)}: {count:,} rows ({percentage:.2f}%)")

if invalid_do_count > 0:
    print(f"\n=== Invalid DOLocationID Values ===")
    print(f"Invalid DOLocationID entries: {invalid_do_count:,}")
    print(f"Percentage of invalid entries: {(invalid_do_count / initial_row_count) * 100:.2f}%")
    
    # Show invalid values (limited to first 20)
    invalid_do_values = df.loc[invalid_do, 'DOLocationID'].value_counts().sort_index().head(20)
    print(f"\nInvalid DOLocationID values found (showing up to 20):")
    for location_id, count in invalid_do_values.items():
        print(f"  LocationID {location_id}: {count:,} rows")
    
    # Drop invalid DOLocationIDs
    df = df[~invalid_do].copy()
    print(f"\nDropped invalid DOLocationID entries.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("\nNo invalid DOLocationID entries found.")

print("*******************************************")

=== PULocationID (Pickup Location) Validation ===
Rows before validation: 3,458,114
Top 10 pickup locations:
  LocationID 161: 157,397 rows (4.55%)
  LocationID 237: 154,285 rows (4.46%)
  LocationID 236: 144,699 rows (4.18%)
  LocationID 132: 120,772 rows (3.49%)
  LocationID 230: 111,855 rows (3.23%)
  LocationID 186: 111,167 rows (3.21%)
  LocationID 162: 111,070 rows (3.21%)
  LocationID 142: 102,272 rows (2.96%)
  LocationID 234: 99,776 rows (2.89%)
  LocationID 170: 94,134 rows (2.72%)

No invalid PULocationID entries found.
*******************************************
=== DOLocationID (Dropoff Location) Validation ===
Top 10 dropoff locations:
  LocationID 236: 150,103 rows (4.34%)
  LocationID 237: 139,696 rows (4.04%)
  LocationID 161: 121,449 rows (3.51%)
  LocationID 230: 99,573 rows (2.88%)
  LocationID 170: 99,268 rows (2.87%)
  LocationID 239: 92,048 rows (2.66%)
  LocationID 68: 91,234 rows (2.64%)
  LocationID 142: 90,960 rows (2.63%)
  LocationID 162: 89,455 rows (2.59%

payment_type

In [12]:
# Allowed payment_type values (only normal transactions: Flex Fare, Credit card, Cash)
allowed_payment_types = [0, 1, 2]

# Check for NaN values first and set to default
nan_count = df['payment_type'].isna().sum()
if nan_count > 0:
    print("=== Payment Type Missing Values ===")
    print(f"NaN/Missing payment_type: {nan_count:,} rows ({(nan_count / len(df)) * 100:.2f}%)")
    df['payment_type'] = df['payment_type'].fillna(1)
    print(f"Set missing payment_type values to default of 1 (Credit card).\n")

# Create boolean mask for invalid payment_types (after filling NaN)
invalid_payment = ~df['payment_type'].isin(allowed_payment_types)
invalid_count = invalid_payment.sum()

# Show overview of all payment_type values before filtering
print("=== Payment Type Overview (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")

print(f"\nAll payment_type distribution:")
all_values = df['payment_type'].value_counts().sort_index()
for payment_type, count in all_values.items():
    percentage = (count / len(df)) * 100
    # Payment type descriptions
    payment_desc = {
        0: "Flex Fare trip",
        1: "Credit card",
        2: "Cash",
        3: "No charge",
        4: "Dispute",
        5: "Unknown",
        6: "Voided trip"
    }
    desc = payment_desc.get(int(payment_type), "Unknown")
    status = "✓ Valid" if int(payment_type) in allowed_payment_types else "✗ To be dropped"
    print(f"  Type {int(payment_type)} ({desc}): {count:,} rows ({percentage:.2f}%) [{status}]")

if invalid_count > 0:
    print(f"\n=== Invalid/Unwanted Payment Types ===")
    print(f"Payment types to drop (3, 4, 5, 6): {invalid_count:,}")
    print(f"Percentage of entries to drop: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Show which payment types are being dropped
    invalid_values = df.loc[invalid_payment, 'payment_type'].value_counts().sort_index()
    print(f"\nPayment types being dropped:")
    for payment_type, count in invalid_values.items():
        percentage = (count / len(df)) * 100
        payment_desc = {
            3: "No charge",
            4: "Dispute",
            5: "Unknown",
            6: "Voided trip"
        }
        desc = payment_desc.get(int(payment_type), "Unknown")
        print(f"  Type {int(payment_type)} ({desc}): {count:,} rows ({percentage:.2f}%)")
    
    # Drop invalid/unwanted payment_types
    df = df[~invalid_payment].copy()
    
    print(f"\nDropped payment types 3, 4, 5, 6.")
    print(f"Rows remaining: {len(df):,}")
    
    # Show final distribution
    print(f"\n=== Payment Type Overview (After Filtering) ===")
    final_values = df['payment_type'].value_counts().sort_index()
    for payment_type, count in final_values.items():
        percentage = (count / len(df)) * 100
        payment_desc = {
            0: "Flex Fare trip",
            1: "Credit card",
            2: "Cash"
        }
        desc = payment_desc.get(int(payment_type), "Unknown")
        print(f"  Type {int(payment_type)} ({desc}): {count:,} rows ({percentage:.2f}%)")
else:
    print("\nNo invalid/unwanted payment types found.")

print("*******************************************")

=== Payment Type Overview (Before Filtering) ===
Rows before validation: 3,458,114

All payment_type distribution:
  Type 0 (Flex Fare trip): 741,846 rows (21.45%) [✓ Valid]
  Type 1 (Credit card): 2,307,575 rows (66.73%) [✓ Valid]
  Type 2 (Cash): 327,262 rows (9.46%) [✓ Valid]
  Type 3 (No charge): 16,079 rows (0.46%) [✗ To be dropped]
  Type 4 (Dispute): 65,352 rows (1.89%) [✗ To be dropped]

=== Invalid/Unwanted Payment Types ===
Payment types to drop (3, 4, 5, 6): 81,431
Percentage of entries to drop: 2.28%

Payment types being dropped:
  Type 3 (No charge): 16,079 rows (0.46%)
  Type 4 (Dispute): 65,352 rows (1.89%)

Dropped payment types 3, 4, 5, 6.
Rows remaining: 3,376,683

=== Payment Type Overview (After Filtering) ===
  Type 0 (Flex Fare trip): 741,846 rows (21.97%)
  Type 1 (Credit card): 2,307,575 rows (68.34%)
  Type 2 (Cash): 327,262 rows (9.69%)
*******************************************


fare_amount

In [13]:
# Invalid fare_amount: NaN, zero, or > $10,000
# Negative values will be converted to positive
# Create boolean masks for each validation check
is_nan = df['fare_amount'].isna()
is_zero = df['fare_amount'] == 0
is_negative = df['fare_amount'] < 0
is_too_high = df['fare_amount'] > 10000

# Show fare statistics BEFORE filtering
print("=== Fare Amount Statistics (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")
print(f"Mean fare: ${df['fare_amount'].mean():.2f}")
print(f"Median fare: ${df['fare_amount'].median():.2f}")
print(f"Min fare: ${df['fare_amount'].min():.2f}")
print(f"Max fare: ${df['fare_amount'].max():.2f}")
print(f"Std deviation: ${df['fare_amount'].std():.2f}")

# Convert negative values to positive
negative_count = is_negative.sum()
if negative_count > 0:
    print(f"\n=== Converting Negative Fare Amounts ===")
    print(f"Negative fare_amount entries: {negative_count:,}")
    df.loc[is_negative, 'fare_amount'] = df.loc[is_negative, 'fare_amount'].abs()
    print(f"Converted {negative_count:,} negative values to positive.\n")

# Combine invalid conditions (excluding negative since we fixed them)
all_invalid = is_nan | is_zero | is_too_high
invalid_count = all_invalid.sum()

if invalid_count > 0:
    print(f"=== Fare Amount Validation Report ===")
    print(f"NaN fare_amount entries: {is_nan.sum():,}")
    print(f"Zero fare_amount entries: {is_zero.sum():,}")
    print(f"Fare_amount > $10,000 entries: {is_too_high.sum():,}")
    print(f"\nTotal invalid fare_amount entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid fare_amount entries
    df = df[~all_invalid].copy()
    
    print(f"\nDropped invalid fare_amount entries.")
    print(f"Rows remaining: {len(df):,}")
    
    # Show fare statistics AFTER filtering
    print(f"\n=== Fare Amount Statistics (After Filtering) ===")
    print(f"Mean fare: ${df['fare_amount'].mean():.2f}")
    print(f"Median fare: ${df['fare_amount'].median():.2f}")
    print(f"Min fare: ${df['fare_amount'].min():.2f}")
    print(f"Max fare: ${df['fare_amount'].max():.2f}")
    print(f"Std deviation: ${df['fare_amount'].std():.2f}")
else:
    print("\nNo invalid fare_amount entries found.")
    
    # Show fare statistics AFTER conversion
    print(f"\n=== Fare Amount Statistics (After Conversion) ===")
    print(f"Mean fare: ${df['fare_amount'].mean():.2f}")
    print(f"Median fare: ${df['fare_amount'].median():.2f}")
    print(f"Min fare: ${df['fare_amount'].min():.2f}")
    print(f"Max fare: ${df['fare_amount'].max():.2f}")
    print(f"Std deviation: ${df['fare_amount'].std():.2f}")

# print highest fare_amount entries
highest_fares = df['fare_amount'].sort_values(ascending=False).head(10)
print("\nTop 10 highest fare_amount entries:")
for fare in highest_fares:
    print(f"  ${fare:.2f}") 

print("*******************************************")

=== Fare Amount Statistics (Before Filtering) ===
Rows before validation: 3,376,683
Mean fare: $17.06
Median fare: $12.80
Min fare: $-653.30
Max fare: $1173.40
Std deviation: $15.75

=== Converting Negative Fare Amounts ===
Negative fare_amount entries: 126,600
Converted 126,600 negative values to positive.

=== Fare Amount Validation Report ===
NaN fare_amount entries: 0
Zero fare_amount entries: 448
Fare_amount > $10,000 entries: 0

Total invalid fare_amount entries: 448
Percentage of invalid entries: 0.01%

Dropped invalid fare_amount entries.
Rows remaining: 3,376,235

=== Fare Amount Statistics (After Filtering) ===
Mean fare: $17.51
Median fare: $12.80
Min fare: $0.01
Max fare: $1173.40
Std deviation: $15.24

Top 10 highest fare_amount entries:
  $1173.40
  $773.00
  $724.00
  $700.00
  $682.70
  $653.30
  $653.30
  $615.50
  $607.80
  $600.00
*******************************************


extra

In [14]:
# Invalid extra: NaN, zero, or > $10,000
# Negative values will be converted to positive
# Create boolean masks for each validation check
is_nan = df['extra'].isna()
is_zero = df['extra'] == 0
is_negative = df['extra'] < 0
is_too_high = df['extra'] > 10000

# Show extra statistics BEFORE filtering
print("=== Extra Amount Statistics (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")
print(f"Mean extra: ${df['extra'].mean():.2f}")
print(f"Median extra: ${df['extra'].median():.2f}")
print(f"Min extra: ${df['extra'].min():.2f}")
print(f"Max extra: ${df['extra'].max():.2f}")
print(f"Std deviation: ${df['extra'].std():.2f}")

# Convert negative values to positive
negative_count = is_negative.sum()
if negative_count > 0:
    print(f"\n=== Converting Negative Extra Amounts ===")
    print(f"Negative extra entries: {negative_count:,}")
    df.loc[is_negative, 'extra'] = df.loc[is_negative, 'extra'].abs()
    print(f"Converted {negative_count:,} negative values to positive.\n")

# Combine invalid conditions (excluding negative since we fixed them)
all_invalid = is_nan | is_too_high
invalid_count = all_invalid.sum()

if invalid_count > 0:
    print(f"=== Extra Amount Validation Report ===")
    print(f"NaN extra entries: {is_nan.sum():,}")
    print(f"Zero extra entries: {is_zero.sum():,}")
    print(f"extra > $10,000 entries: {is_too_high.sum():,}")
    print(f"\nTotal invalid extra entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid fare_amount entries
    df = df[~all_invalid].copy()
    
    print(f"\nDropped invalid extra entries.")
    print(f"Rows remaining: {len(df):,}")
    
    # Show fare statistics AFTER filtering
    print(f"\n=== Extra Amount Statistics (After Filtering) ===")
    print(f"Mean extra: ${df['extra'].mean():.2f}")
    print(f"Median extra: ${df['extra'].median():.2f}")
    print(f"Min extra: ${df['extra'].min():.2f}")
    print(f"Max extra: ${df['extra'].max():.2f}")
    print(f"Std deviation: ${df['extra'].std():.2f}")
else:
    print("\nNo invalid extra entries found.")
    
    # Show fare statistics AFTER conversion
    print(f"\n=== Extra Amount Statistics (After Conversion) ===")
    print(f"Mean extra: ${df['extra'].mean():.2f}")
    print(f"Median extra: ${df['extra'].median():.2f}")
    print(f"Min extra: ${df['extra'].min():.2f}")
    print(f"Max extra: ${df['extra'].max():.2f}")
    print(f"Std deviation: ${df['extra'].std():.2f}")

# print highest extra entries
highest_extras = df['extra'].sort_values(ascending=False).head(10)
print("\nTop 10 highest extra entries:")
for extra in highest_extras:
    print(f"  ${extra:.2f}") 

print("*******************************************")



=== Extra Amount Statistics (Before Filtering) ===
Rows before validation: 3,376,235
Mean extra: $1.29
Median extra: $0.00
Min extra: $-7.50
Max extra: $13.50
Std deviation: $1.84

=== Converting Negative Extra Amounts ===
Negative extra entries: 5,998
Converted 5,998 negative values to positive.


No invalid extra entries found.

=== Extra Amount Statistics (After Conversion) ===
Mean extra: $1.30
Median extra: $0.00
Min extra: $0.00
Max extra: $13.50
Std deviation: $1.83

Top 10 highest extra entries:
  $13.50
  $13.25
  $13.25
  $13.25
  $12.50
  $12.50
  $12.50
  $12.50
  $12.50
  $12.50
*******************************************


mta_tax

In [15]:
# Mobilitätssteuer für den städtischen Pendlerverkehr, 0,50 $ pro Fahrt in NYC für bestimmte dropoff-Ziele.
# Validierung der dropoff-Ziele wird ignoriert!
# Invalid mta_tax: NaN, zero, or > $10,000
# Negative values will be converted to positive
# Create boolean masks for each validation check
is_nan = df['mta_tax'].isna()
# valid mta_tax is exactly 0.5 or zero
is_valid = df['mta_tax'].isin([0, 0.5])
is_negative = df['mta_tax'] < 0

# print all unique mta_tax values before filtering with counts
unique_mta_taxes = df['mta_tax'].unique()
print("=== Unique mta_tax Values (Before Filtering) ===")
for mta_tax in sorted(unique_mta_taxes):
    print(f"  ${mta_tax:.2f}: {df[df['mta_tax'] == mta_tax].shape[0]:,} rows")
print("*******************************************")

# Show mta_tax statistics BEFORE filtering
print("=== mta_tax Amount Statistics (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")
print(f"Mean mta_tax: ${df['mta_tax'].mean():.2f}")
print(f"Median mta_tax: ${df['mta_tax'].median():.2f}")
print(f"Min mta_tax: ${df['mta_tax'].min():.2f}")
print(f"Max mta_tax: ${df['mta_tax'].max():.2f}")
print(f"Std deviation: ${df['mta_tax'].std():.2f}")

# Convert negative values to positive
negative_count = is_negative.sum()
if negative_count > 0:
    print(f"\n=== Converting Negative mta_tax Amounts ===")
    print(f"Negative mta_tax entries: {negative_count:,}")
    df.loc[is_negative, 'mta_tax'] = df.loc[is_negative, 'mta_tax'].abs()
    print(f"Converted {negative_count:,} negative values to positive.\n")

# Combine invalid conditions (excluding negative since we fixed them)
all_invalid = is_nan | ~is_valid
invalid_count = all_invalid.sum()

if invalid_count > 0:
    print(f"=== mta_tax Amount Validation Report ===")
    print(f"NaN mta_tax entries: {is_nan.sum():,}")
    print(f"Zero mta_tax entries: {is_zero.sum():,}")
    print(f"mta_tax != $0.50 entries: {~is_valid.sum():,}")
    print(f"\nTotal invalid mta_tax entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid mta_tax entries
    df = df[~all_invalid].copy()
    
    print(f"\nDropped invalid mta_tax entries.")
    print(f"Rows remaining: {len(df):,}")
    
    # Show mta_tax statistics AFTER filtering
    print(f"\n=== mta_tax Amount Statistics (After Filtering) ===")
    print(f"Mean mta_tax: ${df['mta_tax'].mean():.2f}")
    print(f"Median mta_tax: ${df['mta_tax'].median():.2f}")
    print(f"Min mta_tax: ${df['mta_tax'].min():.2f}")
    print(f"Max mta_tax: ${df['mta_tax'].max():.2f}")
    print(f"Std deviation: ${df['mta_tax'].std():.2f}")
else:
    print("\nNo invalid mta_tax entries found.")
    
    # Show mta_tax statistics AFTER conversion
    print(f"\n=== mta_tax Amount Statistics (After Conversion) ===")
    print(f"Mean mta_tax: ${df['mta_tax'].mean():.2f}")
    print(f"Median mta_tax: ${df['mta_tax'].median():.2f}")
    print(f"Min mta_tax: ${df['mta_tax'].min():.2f}")
    print(f"Max mta_tax: ${df['mta_tax'].max():.2f}")
    print(f"Std deviation: ${df['mta_tax'].std():.2f}")

# print highest mta_tax entries
highest_mta_taxes = df['mta_tax'].sort_values(ascending=False).head(10)
print("\nTop 10 highest mta_tax entries:")
for mta_tax in highest_mta_taxes:
    print(f"  ${mta_tax:.2f}")


# print numbers of values in mta_tax
mta_tax_counts = df['mta_tax'].value_counts().sort_index()
print("\n=== mta_tax Value Counts ===")
for mta_tax, count in mta_tax_counts.items():
    print(f"  mta_tax ${mta_tax:.2f}: {count:,} rows")  
print("*******************************************")



=== Unique mta_tax Values (Before Filtering) ===
  $-0.50: 11,389 rows
  $0.00: 18,573 rows
  $0.50: 3,346,212 rows
  $1.00: 52 rows
  $1.75: 1 rows
  $2.25: 1 rows
  $4.00: 1 rows
  $4.75: 4 rows
  $10.50: 2 rows
*******************************************
=== mta_tax Amount Statistics (Before Filtering) ===
Rows before validation: 3,376,235
Mean mta_tax: $0.49
Median mta_tax: $0.50
Min mta_tax: $-0.50
Max mta_tax: $10.50
Std deviation: $0.07

=== Converting Negative mta_tax Amounts ===
Negative mta_tax entries: 11,389
Converted 11,389 negative values to positive.

=== mta_tax Amount Validation Report ===
NaN mta_tax entries: 0
Zero mta_tax entries: 1,799,248
mta_tax != $0.50 entries: -3,364,786

Total invalid mta_tax entries: 11,450
Percentage of invalid entries: 0.32%

Dropped invalid mta_tax entries.
Rows remaining: 3,364,785

=== mta_tax Amount Statistics (After Filtering) ===
Mean mta_tax: $0.50
Median mta_tax: $0.50
Min mta_tax: $0.00
Max mta_tax: $0.50
Std deviation: $0.04

Top

tip_amount

In [16]:
# Tip amount validation
# Tips can be $0 (cash payments don't record tips) or positive amounts
# Negative values will be converted to positive
# NaN values will be set to 0
# Extremely high tips (> $1000) will be dropped as outliers

# Create boolean masks for each validation check
is_nan = df['tip_amount'].isna()
is_negative = df['tip_amount'] < 0
is_too_high = df['tip_amount'] > 1000

# Show tip_amount statistics BEFORE filtering
print("=== Tip Amount Statistics (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")
print(f"Mean tip: ${df['tip_amount'].mean():.2f}")
print(f"Median tip: ${df['tip_amount'].median():.2f}")
print(f"Min tip: ${df['tip_amount'].min():.2f}")
print(f"Max tip: ${df['tip_amount'].max():.2f}")
print(f"Std deviation: ${df['tip_amount'].std():.2f}")

# Handle NaN values - set to 0 (no tip recorded)
nan_count = is_nan.sum()
if nan_count > 0:
    print(f"\n=== Handling Missing Tip Amounts ===")
    print(f"NaN tip_amount entries: {nan_count:,}")
    df['tip_amount'] = df['tip_amount'].fillna(0)
    print(f"Set missing tip_amount values to $0.00.")

# Convert negative values to positive
negative_count = is_negative.sum()
if negative_count > 0:
    print(f"\n=== Converting Negative Tip Amounts ===")
    print(f"Negative tip_amount entries: {negative_count:,}")
    df.loc[is_negative, 'tip_amount'] = df.loc[is_negative, 'tip_amount'].abs()
    print(f"Converted {negative_count:,} negative values to positive.")

# Drop extremely high tips (> $1000)
invalid_count = is_too_high.sum()
if invalid_count > 0:
    print(f"\n=== Tip Amount Validation Report ===")
    print(f"Tip_amount > $1,000 entries: {invalid_count:,}")
    print(f"Percentage of invalid entries: {(invalid_count / initial_row_count) * 100:.2f}%")
    
    # Drop invalid tip_amount entries
    df = df[~is_too_high].copy()
    
    print(f"Dropped invalid tip_amount entries.")
    print(f"Rows remaining: {len(df):,}")

# Show tip statistics AFTER filtering/conversion
print(f"\n=== Tip Amount Statistics (After Filtering) ===")
print(f"Mean tip: ${df['tip_amount'].mean():.2f}")
print(f"Median tip: ${df['tip_amount'].median():.2f}")
print(f"Min tip: ${df['tip_amount'].min():.2f}")
print(f"Max tip: ${df['tip_amount'].max():.2f}")
print(f"Std deviation: ${df['tip_amount'].std():.2f}")

# Show highest tip_amount entries
highest_tips = df.nlargest(10, 'tip_amount')['tip_amount']
print("\n=== Top 10 Highest Tips ===")
for idx, tip in enumerate(highest_tips, 1):
    print(f"  {idx}. ${tip:.2f}")

# Show distribution of tip amounts
print("\n=== Tip Amount Distribution ===")
no_tip_count = (df['tip_amount'] == 0).sum()
with_tip_count = (df['tip_amount'] > 0).sum()
print(f"Trips with no tip ($0): {no_tip_count:,} ({(no_tip_count / len(df)) * 100:.2f}%)")
print(f"Trips with tips: {with_tip_count:,} ({(with_tip_count / len(df)) * 100:.2f}%)")

# Show tip amount per payment type
print("\n=== Tip Amount by Payment Type ===")
payment_type_desc = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash"
}

for payment_type in allowed_payment_types:
    desc = payment_type_desc.get(payment_type, "Unknown")
    payment_mask = df['payment_type'] == payment_type
    total_tips = df.loc[payment_mask, 'tip_amount'].sum()
    trip_count = payment_mask.sum()
    avg_tip = df.loc[payment_mask, 'tip_amount'].mean()
    
    print(f"  Payment Type {payment_type} ({desc}):")
    print(f"    Total Tips: ${total_tips:,.2f}")
    print(f"    Number of Trips: {trip_count:,}")
    print(f"    Average Tip per Trip: ${avg_tip:.2f}")

# Drop rows with payment_type 2 (Cash) and tip_amount > 0
# Cash tips are not recorded in the system, so this indicates a data error
print("\n=== Cash Payment Tip Validation ===")
cash_with_tips = (df['payment_type'] == 2) & (df['tip_amount'] > 0)
cash_with_tips_count = cash_with_tips.sum()

if cash_with_tips_count > 0:
    print(f"Cash payments with recorded tips (data error): {cash_with_tips_count:,}")
    print(f"Percentage: {(cash_with_tips_count / len(df)) * 100:.2f}%")
    df = df[~cash_with_tips].copy()
    print(f"Dropped cash trips with tips > $0.")
    print(f"Rows remaining: {len(df):,}")
else:
    print("No cash payments with tips found (expected behavior).")

print("*******************************************")

=== Tip Amount Statistics (Before Filtering) ===
Rows before validation: 3,364,785
Mean tip: $2.85
Median tip: $2.35
Min tip: $0.00
Max tip: $440.00
Std deviation: $3.61

=== Tip Amount Statistics (After Filtering) ===
Mean tip: $2.85
Median tip: $2.35
Min tip: $0.00
Max tip: $440.00
Std deviation: $3.61

=== Top 10 Highest Tips ===
  1. $440.00
  2. $225.00
  3. $220.00
  4. $180.30
  5. $180.00
  6. $171.07
  7. $166.00
  8. $150.00
  9. $146.39
  10. $128.00

=== Tip Amount Distribution ===
Trips with no tip ($0): 1,137,384 (33.80%)
Trips with tips: 2,227,401 (66.20%)

=== Tip Amount by Payment Type ===
  Payment Type 0 (Flex Fare trip):
    Total Tips: $237,960.41
    Number of Trips: 741,772
    Average Tip per Trip: $0.32
  Payment Type 1 (Credit card):
    Total Tips: $9,349,520.33
    Number of Trips: 2,307,534
    Average Tip per Trip: $4.05
  Payment Type 2 (Cash):
    Total Tips: $259.62
    Number of Trips: 315,479
    Average Tip per Trip: $0.00

=== Cash Payment Tip Valid

tolls_amount

In [17]:
# Tolls amount validation
# Tolls can be $0 (no tolls) or positive amounts
# Negative values will be converted to positive
# NaN values will be set to 0
# Extremely high tolls (> $100) will be flagged and potentially dropped

# Create boolean masks for each validation check
is_nan = df['tolls_amount'].isna()
is_negative = df['tolls_amount'] < 0
is_too_high = df['tolls_amount'] > 100

# Show tolls_amount statistics BEFORE filtering
print("=== Tolls Amount Statistics (Before Filtering) ===")
print(f"Rows before validation: {len(df):,}")
print(f"Mean tolls: ${df['tolls_amount'].mean():.2f}")
print(f"Median tolls: ${df['tolls_amount'].median():.2f}")
print(f"Min tolls: ${df['tolls_amount'].min():.2f}")
print(f"Max tolls: ${df['tolls_amount'].max():.2f}")
print(f"Std deviation: ${df['tolls_amount'].std():.2f}")

# Handle NaN values - set to 0 (no tolls)
nan_count = is_nan.sum()
if nan_count > 0:
    print(f"\n=== Handling Missing Tolls Amounts ===")
    print(f"NaN tolls_amount entries: {nan_count:,} ({(nan_count / len(df)) * 100:.2f}%)")
    df['tolls_amount'] = df['tolls_amount'].fillna(0)
    print(f"Set missing tolls_amount values to $0.00.")

# Convert negative values to positive
negative_count = is_negative.sum()
if negative_count > 0:
    print(f"\n=== Converting Negative Tolls Amounts ===")
    print(f"Negative tolls_amount entries: {negative_count:,} ({(negative_count / len(df)) * 100:.2f}%)")
    df.loc[is_negative, 'tolls_amount'] = df.loc[is_negative, 'tolls_amount'].abs()
    print(f"Converted {negative_count:,} negative values to positive.")

# Check for extremely high tolls (> $100)
high_tolls_count = is_too_high.sum()
if high_tolls_count > 0:
    print(f"\n=== High Tolls Amount Review ===")
    print(f"Tolls_amount > $100 entries: {high_tolls_count:,} ({(high_tolls_count / len(df)) * 100:.2f}%)")
    
    # Show the high toll amounts
    high_tolls = df[is_too_high]['tolls_amount'].value_counts().sort_index()
    print(f"\nHigh toll amounts found:")
    for toll_amount, count in high_tolls.head(10).items():
        print(f"  ${toll_amount:.2f}: {count:,} trips")
    
    # Drop extremely high tolls (likely data errors)
    df = df[~is_too_high].copy()
    print(f"\nDropped tolls > $100 as outliers.")
    print(f"Rows remaining: {len(df):,}")

# Show tolls statistics AFTER filtering/conversion
print(f"\n=== Tolls Amount Statistics (After Filtering) ===")
print(f"Mean tolls: ${df['tolls_amount'].mean():.2f}")
print(f"Median tolls: ${df['tolls_amount'].median():.2f}")
print(f"Min tolls: ${df['tolls_amount'].min():.2f}")
print(f"Max tolls: ${df['tolls_amount'].max():.2f}")
print(f"Std deviation: ${df['tolls_amount'].std():.2f}")

# Show distribution of toll amounts
print(f"\n=== Tolls Amount Distribution ===")
no_tolls = (df['tolls_amount'] == 0).sum()
with_tolls = (df['tolls_amount'] > 0).sum()
print(f"Trips with no tolls ($0): {no_tolls:,} ({(no_tolls / len(df)) * 100:.2f}%)")
print(f"Trips with tolls: {with_tolls:,} ({(with_tolls / len(df)) * 100:.2f}%)")

# Show top 20 most common toll amounts (excluding $0)
if with_tolls > 0:
    print(f"\n=== Top 20 Most Common Toll Amounts (excluding $0) ===")
    tolls_nonzero = df[df['tolls_amount'] > 0]['tolls_amount'].value_counts().head(20)
    for idx, (toll_amount, count) in enumerate(tolls_nonzero.items(), 1):
        percentage = (count / with_tolls) * 100
        print(f"  {idx}. ${toll_amount:.2f}: {count:,} trips ({percentage:.2f}% of trips with tolls)")

print("*******************************************")

=== Tolls Amount Statistics (Before Filtering) ===
Rows before validation: 3,364,753
Mean tolls: $0.42
Median tolls: $0.00
Min tolls: $-48.18
Max tolls: $115.87
Std deviation: $1.87

=== Converting Negative Tolls Amounts ===
Negative tolls_amount entries: 123 (0.00%)
Converted 123 negative values to positive.

=== High Tolls Amount Review ===
Tolls_amount > $100 entries: 3 (0.00%)

High toll amounts found:
  $101.94: 1 trips
  $113.93: 1 trips
  $115.87: 1 trips

Dropped tolls > $100 as outliers.
Rows remaining: 3,364,750

=== Tolls Amount Statistics (After Filtering) ===
Mean tolls: $0.42
Median tolls: $0.00
Min tolls: $0.00
Max tolls: $92.06
Std deviation: $1.87

=== Tolls Amount Distribution ===
Trips with no tolls ($0): 3,174,521 (94.35%)
Trips with tolls: 190,229 (5.65%)

=== Top 20 Most Common Toll Amounts (excluding $0) ===
  1. $6.94: 174,912 trips (91.95% of trips with tolls)
  2. $14.06: 2,395 trips (1.26% of trips with tolls)
  3. $16.06: 1,816 trips (0.95% of trips with tol

improvement_surcharge

In [18]:
# print NaN from improvement_surcharge
print("Number of NaN entries in improvement_surcharge:", df['improvement_surcharge'].isna().sum())
# print negative improvement_surcharge entries
print("Number of negative improvement_surcharge entries:", df[df['improvement_surcharge'] < 0].shape[0])   
# print value counts 
print("Value counts for improvement_surcharge:")
print(df['improvement_surcharge'].value_counts())

Number of NaN entries in improvement_surcharge: 0
Number of negative improvement_surcharge entries: 285
Value counts for improvement_surcharge:
improvement_surcharge
 1.0    3329740
 0.0      34561
-1.0        285
 0.3        164
Name: count, dtype: int64


total_amount

In [19]:
# print NaN from total_amount
print("Number of NaN entries in total_amount:", df['total_amount'].isna().sum())
# print negative total_amount entries
print("Number of negative total_amount entries:", df[df['total_amount'] < 0].shape[0])  

Number of NaN entries in total_amount: 0
Number of negative total_amount entries: 285


congestion_surcharge

In [20]:
# print NaN from congestion_surcharge
print("Number of NaN entries in congestion_surcharge:", df['congestion_surcharge'].isna().sum())
# print negative congestion_surcharge entries
print("Number of negative congestion_surcharge entries:", df[df['congestion_surcharge'] < 0].shape[0])
# print value counts
print("Value counts for congestion_surcharge:")
print(df['congestion_surcharge'].value_counts())

Number of NaN entries in congestion_surcharge: 741772
Number of negative congestion_surcharge entries: 99
Value counts for congestion_surcharge:
congestion_surcharge
 2.5    2450684
 0.0     172195
-2.5         99
Name: count, dtype: int64


Airport_fee (field name case sensitive!!!)

In [21]:
# print NaN from airport_fee
print("Number of NaN entries in airport_fee:", df['Airport_fee'].isna().sum())
# print negative airport_fee entries
print("Number of negative airport_fee entries:", df[df['Airport_fee'] < 0].shape[0])
# print value counts
print("Value counts for airport_fee:")
print(df['Airport_fee'].value_counts())


Number of NaN entries in airport_fee: 741772
Number of negative airport_fee entries: 151
Value counts for airport_fee:
Airport_fee
 0.00    2431913
 1.75     190914
-1.75        151
Name: count, dtype: int64


cbd_congestion_fee

In [22]:
# print NaN from cbd_congestion_fee
print("Number of NaN entries in cbd_congestion_fee:", df['cbd_congestion_fee'].isna().sum())
# print negative cbd_congestion_fee entries
print("Number of negative cbd_congestion_fee entries:", df[df['cbd_congestion_fee'] < 0].shape[0])

Number of NaN entries in cbd_congestion_fee: 0
Number of negative cbd_congestion_fee entries: 168


Duplicate Rows

In [23]:
# duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())
print("percentage of duplicate rows before dropping duplicates: ", (df.duplicated().sum() / initial_row_count) * 100)
df = df.drop_duplicates()
print(f"Number of rows in the dataframe after dropping duplicates: {len(df)}")

Number of duplicate rows: 0
percentage of duplicate rows before dropping duplicates:  0.0
Number of rows in the dataframe after dropping duplicates: 3364750


CLEANING DONE

In [24]:

print("*******************************************")
print("***** COMPLETED CLEANING *****")
# dropped rows initial vs final 
print(f"Initial Number of rows in the dataframe: {initial_row_count:,.0f}")

dropped_rows = initial_row_count - len(df)
print(f"Number of rows dropped: {dropped_rows:,.0f}")

print(f"Final Number of rows in the dataframe: {len(df):,.0f}")  

# percentage of rows dropped
percentage_dropped = (dropped_rows / initial_row_count) * 100
print(f"Percentage of rows dropped: {percentage_dropped:.2f}%")


*******************************************
***** COMPLETED CLEANING *****
Initial Number of rows in the dataframe: 3,577,543
Number of rows dropped: 212,793
Final Number of rows in the dataframe: 3,364,750
Percentage of rows dropped: 5.95%


Tip per person

In [25]:
# tip per passenger count
df['tip_per_passenger'] = df['tip_amount'] / df['passenger_count']
#print(df.head(5))

# print total tip_amount 
total_tip_amount = df['tip_amount'].sum()
print(f"Total tip amount: ${total_tip_amount:,.2f}")

# print total passenger count (formatting with commas)
total_passenger_count = df['passenger_count'].sum()
print(f"Total passenger count: {total_passenger_count:,}")
# average tip per passenger
average_tip_per_passenger = total_tip_amount / total_passenger_count
print(f"Average tip per passenger: ${average_tip_per_passenger:.2f}")




Total tip amount: $9,587,373.39
Total passenger count: 4,102,387.0
Average tip per passenger: $2.34


Umsatz pro Weglänge (Verhältnis)

In [26]:
# print revenue per mile

Gesamtumsatz pro Taximeter (vendor_id)

In [27]:
# mapping of VendorID to VendorName for better readability
vendor_id_map = {1: 'Creative Mobile Technologies, LLC', 2: 'Curb Mobility, LLC', 6: 'Myle Technologies Inc', 7: 'Helix'}
df['VendorName'] = df['VendorID'].map(vendor_id_map)
# revenue per VendorName formatting in USD with 2 decimal places
revenue_per_vendor = df.groupby('VendorName')['total_amount'].sum()
revenue_per_vendor = revenue_per_vendor.apply(lambda x: f"${x:,.2f}")
print(revenue_per_vendor)


VendorName
Creative Mobile Technologies, LLC    $18,713,698.43
Curb Mobility, LLC                   $68,066,486.11
Myle Technologies Inc                     $5,327.58
Name: total_amount, dtype: object


Median von Weglänge aller Fahrten pro Tag und Stunde

Visualisieren